##**Mounting Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Importing Libraries**

In [2]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.6 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

#Hyperparameter Optimization
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-3-ff41bca45932>:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


##**Determining File Path and Separating into Subfolders**

In [4]:
data_dir = '/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/'

data_path = "/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/"
glaucoma_path = os.path.join(data_path, "Glaucoma")
normal_path = os.path.join(data_path, "Normal")

print(os.listdir("/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/"))

['Glaucoma', 'Normal', 'Cataract', 'Diabetic Retinopathy']


##**Specifying Image Size**

In [5]:
img_height, img_width = 128, 128

##**Loading Images**

In [8]:
def load_images(folder_path, label):
    images = []
    labels = []
    for idx, filename in enumerate(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error: Unable to read image {img_path}")
            continue
        img = cv2.resize(img, (img_height, img_width))
        images.append(img)
        labels.append(label)
        print(f"Image {idx + 1} loaded from {img_path}")
    return np.array(images), np.array(labels)

glaucoma_images, glaucoma_labels = load_images(glaucoma_path, 0)
normal_images, normal_labels = load_images(normal_path, 1)

Image 1 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1229_left.jpg
Image 2 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1248_left.jpg
Image 3 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1237_left.jpg
Image 4 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1217_right.jpg
Image 5 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1230_left.jpg
Image 6 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1220_right.jpg
Image 7 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1294_left.jpg
Image 8 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1314_left.jpg
Image 9 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1247_left.jpg
Image 10 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Glaucoma/1217_left.jpg
Image 1

##**Concatenating the Data Set as Cataract and Normal**

In [9]:
x = np.concatenate((glaucoma_images, normal_images), axis=0)
y = np.concatenate((glaucoma_labels, normal_labels), axis=0)

###Displaying the Number of Data Samples for X and Y

In [10]:
print("Number of data samples:", x.shape[0])
print("Number of data samples:", y.shape[0])

Number of data samples: 2081
Number of data samples: 2081


###Splitting the Data Set into Training (80%) and Test (20%) Sets

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 38)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 38)

###Training and Finalizing the CNN Model

In [12]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [13]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [14]:
def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv1', min_value=0.2, max_value=0.5, step=0.1)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 3
    model.add(Dense(units=hp.Int('dense3_units', min_value=64, max_value=256, step=16)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [15]:
# Creating the tuner using RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Maximum number of models to try
    directory='my_dir',  # Directory to store saved models and results
    project_name='retinal_imaging_hyperparameter_tuning'  # Name of the tuning project
)

# Fitting the tuner with training data
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# Getting the best hyperparameters and building the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Training the best model on the training set
best_model.fit(x_train, train_yCl, epochs=15, validation_data=(x_val, valid_yCl))

Trial 5 Complete [00h 01m 16s]
val_accuracy: 0.85628741979599

Best val_accuracy So Far: 0.85628741979599
Total elapsed time: 00h 07m 23s
Epoch 1/15
47/47 [==============================] - 7s 86ms/step - loss: 55.4790 - accuracy: 0.6379 - val_loss: 0.5961 - val_accuracy: 0.8503
Epoch 2/15
47/47 [==============================] - 4s 76ms/step - loss: 0.5419 - accuracy: 0.7762 - val_loss: 0.5144 - val_accuracy: 0.8383
Epoch 3/15
47/47 [==============================] - 4s 76ms/step - loss: 0.4376 - accuracy: 0.8263 - val_loss: 0.4317 - val_accuracy: 0.8084
Epoch 4/15
47/47 [==============================] - 4s 75ms/step - loss: 0.4036 - accuracy: 0.8337 - val_loss: 0.3868 - val_accuracy: 0.8204
Epoch 5/15
47/47 [==============================] - 4s 79ms/step - loss: 0.3856 - accuracy: 0.8504 - val_loss: 0.3796 - val_accuracy: 0.8383
Epoch 6/15
47/47 [==============================] - 4s 75ms/step - loss: 0.3521 - accuracy: 0.8544 - val_loss: 0.3486 - val_accuracy: 0.8443
Epoch 7/15
47/4

In [16]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

6/6 [==============================] - 0s 11ms/step - loss: 0.3598 - accuracy: 0.8323
Validation Accuracy:  0.832335352897644
14/14 [==============================] - 0s 29ms/step - loss: 0.3749 - accuracy: 0.8273
Test Accuracy:  0.8273381590843201
47/47 [==============================] - 1s 12ms/step - loss: 0.3065 - accuracy: 0.8778
Training Accuracy:  0.8777555227279663


In [17]:
# Making predictions on the test data
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Calculating metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))

# Printing the results
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Displaying the Confusion Matrix
print('Confusion Matrix:')
print(conf_matrix)

# Showing the Classification Report
print('Classification Report:')
print(class_report)

14/14 [==============================] - 0s 11ms/step
Test Accuracy: 0.8273
Precision: 0.8437
Recall: 0.8273
F1 Score: 0.8237
Confusion Matrix:
[[136  60]
 [ 12 209]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.69      0.79       196
           1       0.78      0.95      0.85       221

    accuracy                           0.83       417
   macro avg       0.85      0.82      0.82       417
weighted avg       0.84      0.83      0.82       417



###Splitting the Data Set into Training (65%) and Test (35%) Sets

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.35, random_state = 38)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 38)

###Training and Finalizing the CNN Model

In [19]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [20]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [21]:
def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv1', min_value=0.2, max_value=0.5, step=0.1)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 3
    model.add(Dense(units=hp.Int('dense3_units', min_value=64, max_value=256, step=16)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [22]:
# Creating the tuner using RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Maximum number of models to try
    directory='my_dir',  # Directory to store saved models and results
    project_name='retinal_imaging_hyperparameter_tuning'  # Name of the tuning project
)

# Fitting the tuner with training data
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# Getting the best hyperparameters and building the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Training the best model on the training set
best_model.fit(x_train, train_yCl, epochs=15, validation_data=(x_val, valid_yCl))

Reloading Tuner from my_dir/retinal_imaging_hyperparameter_tuning/tuner0.json
Epoch 1/15
38/38 [==============================] - 5s 86ms/step - loss: 125.0923 - accuracy: 0.5855 - val_loss: 0.6404 - val_accuracy: 0.5882
Epoch 2/15
38/38 [==============================] - 3s 76ms/step - loss: 0.5006 - accuracy: 0.7837 - val_loss: 0.3915 - val_accuracy: 0.8824
Epoch 3/15
38/38 [==============================] - 3s 77ms/step - loss: 0.4012 - accuracy: 0.8322 - val_loss: 0.6494 - val_accuracy: 0.7059
Epoch 4/15
38/38 [==============================] - 3s 77ms/step - loss: 0.3930 - accuracy: 0.8388 - val_loss: 0.3563 - val_accuracy: 0.8603
Epoch 5/15
38/38 [==============================] - 3s 76ms/step - loss: 0.3715 - accuracy: 0.8594 - val_loss: 0.3362 - val_accuracy: 0.8824
Epoch 6/15
38/38 [==============================] - 3s 76ms/step - loss: 0.3467 - accuracy: 0.8528 - val_loss: 0.3124 - val_accuracy: 0.8971
Epoch 7/15
38/38 [==============================] - 3s 76ms/step - loss: 0

In [23]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

5/5 [==============================] - 0s 14ms/step - loss: 0.3305 - accuracy: 0.8824
Validation Accuracy:  0.8823529481887817
23/23 [==============================] - 0s 14ms/step - loss: 0.3482 - accuracy: 0.8464
Test Accuracy:  0.8463648557662964
38/38 [==============================] - 0s 11ms/step - loss: 0.2742 - accuracy: 0.8824
Training Accuracy:  0.8824012875556946


In [24]:
# Making predictions on the test data
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Calculating metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))

# Printing the results
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Displaying the Confusion Matrix
print('Confusion Matrix:')
print(conf_matrix)

# Showing the Classification Report
print('Classification Report:')
print(class_report)

23/23 [==============================] - 0s 10ms/step
Test Accuracy: 0.8464
Precision: 0.8630
Recall: 0.8464
F1 Score: 0.8432
Confusion Matrix:
[[243  97]
 [ 15 374]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.71      0.81       340
           1       0.79      0.96      0.87       389

    accuracy                           0.85       729
   macro avg       0.87      0.84      0.84       729
weighted avg       0.86      0.85      0.84       729

